In [23]:
import numpy as np

Specify XYZ of molecule

In [3]:
HeH_xyz = [
['He', (0, 0, 0)],
['H', (0, 0, 1.4632)]
]

In [6]:
N_atoms = len(HeH_xyz)
atom_types = [atm[0] for atm in HeH_xyz]
atom_coord = [atm[1:] for atm in HeH_xyz]

## Specify Basis set

- click link below to find basis sets
- select JASON output!

https://www.basissetexchange.org/

https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Book%3A_Quantum_Mechanics__in_Chemistry_(Simons_and_Nichols)/18%3A_Multiconfiguration_Wavefunctions/18.04%3A_Atomic_Orbital_Basis_Sets

In [16]:
basis_He_H = {
    "molssi_bse_schema": {
        "schema_type": "complete",
        "schema_version": "0.1"
    },
    "revision_description": "Data from Gaussian09",
    "revision_date": "2018-06-19",
    "elements": {
        "1": {
            "electron_shells": [
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0
                    ],
                    "exponents": [
                        "0.3425250914E+01",
                        "0.6239137298E+00",
                        "0.1688554040E+00"
                    ],
                    "coefficients": [
                        [
                            "0.1543289673E+00",
                            "0.5353281423E+00",
                            "0.4446345422E+00"
                        ]
                    ]
                }
            ],
            "references": [
                {
                    "reference_description": "STO-3G Minimal Basis (3 functions/AO)",
                    "reference_keys": [
                        "hehre1969a"
                    ]
                }
            ]
        },
        "2": {
            "electron_shells": [
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0
                    ],
                    "exponents": [
                        "0.6362421394E+01",
                        "0.1158922999E+01",
                        "0.3136497915E+00"
                    ],
                    "coefficients": [
                        [
                            "0.1543289673E+00",
                            "0.5353281423E+00",
                            "0.4446345422E+00"
                        ]
                    ]
                }
            ],
            "references": [
                {
                    "reference_description": "STO-3G Minimal Basis (3 functions/AO)",
                    "reference_keys": [
                        "hehre1969a",
                        "hehre1970a"
                    ]
                }
            ]
        }
    },
    "version": "1",
    "function_types": [
        "gto",
        "gto_spherical"
    ],
    "names": [
        "STO-3G"
    ],
    "tags": [],
    "family": "sto",
    "description": "STO-3G Minimal Basis (3 functions/AO)",
    "role": "orbital",
    "auxiliaries": {},
    "name": "STO-3G"
}


In [19]:
H_basis = basis_He_H['elements']['1']
He_basis = basis_He_H['elements']['2']

In [20]:
H_basis

{'electron_shells': [{'function_type': 'gto',
   'region': '',
   'angular_momentum': [0],
   'exponents': ['0.3425250914E+01', '0.6239137298E+00', '0.1688554040E+00'],
   'coefficients': [['0.1543289673E+00',
     '0.5353281423E+00',
     '0.4446345422E+00']]}],
 'references': [{'reference_description': 'STO-3G Minimal Basis (3 functions/AO)',
   'reference_keys': ['hehre1969a']}]}

['0.1543289673E+00', '0.5353281423E+00', '0.4446345422E+00']

In [38]:
# STO-nG (n = number of Gaussian functions used to form contracted [via linear combination] Gaussian orbital)
STOnG=3

In [39]:
# Gaussian orbital contraction coefficients
D = np.array( 
[
    H_basis['electron_shells'][0]['coefficients'][0],
    He_basis['electron_shells'][0]['coefficients'][0]
], dtype=float
)
D

array([[0.15432897, 0.53532814, 0.44463454],
       [0.15432897, 0.53532814, 0.44463454]])

In [40]:
alpha = np.array( # Gaussian orbital exponents 
[
    H_basis['electron_shells'][0]['exponents'],
    He_basis['electron_shells'][0]['exponents']
], dtype=float
)
alpha

array([[3.42525091, 0.62391373, 0.1688554 ],
       [6.36242139, 1.158923  , 0.31364979]])

In [149]:
## important constant things

max_quantum_number_dict = {'H':1, 'He':1, 'Li':2, 'Be':2, 'C':2} ## etc for all periodic table (max shell number)
N_electrons = 2

atomic_charges_dict = {
    'H':1,
    'He':2,
    'Li':3,
    'Be':4,
    'B':5
} ## etc for all periodic table (proton number)

In [150]:
# Basis set size

Basis_set_size = sum(max_quantum_number_dict[atom_str] for atom_str in atom_types)
Basis_set_size

2

**IMPORTANT: we are only performing integrals for 1s-type primitive Gaussian functions here**
(different approach required for other terms... But overall idea is very similar!)

An un-normalized 1s primitive Gaussian centred at $R_{A}$ is:

$$g_{1s}(r-R_{A}) = e^{-\alpha |r-R_{A}|^{2}}$$

- Using $\alpha, \beta$ for the orbital exponents functions for the functions centered at $R_{A}$ and $R_{B}$ 
- A product of two $1s$ Gaussians is proportional to a SINGLE $1s$ orbital on a different (third) centre $R_{p}$:

$$ \bigg(g_{1s}(r-R_{A})  \bigg) \bigg( g_{1s}(r-R_{B}) \bigg) = \bigg( e^{-\alpha |r-R_{A}|^{2}}\bigg) \bigg(e^{-\beta |r-R_{B}|^{2}}\bigg) = K g_{1s}(r-R_{P})$$


The constant of proportionality $K$ is given by:

$$K = e^{-\frac{\alpha \beta}{(\alpha + \beta)} |R_{A}-R_{B}|^{2}}$$

The new centre is $R_{P}$ is:

$$R_{P} = \frac{\alpha R_{A} + \beta R_{B}}{(\alpha + \beta)}$$

The exponent of the new orbital is:

$$p = \alpha + \beta$$

In [114]:
def Product_of_two_Gaussians(GaussA, GaussB):
    alpha, R_A = GaussA
    beta, R_B = GaussB
    
    R_A = np.asarray(R_A).flatten()
    R_B = np.asarray(R_B).flatten()
    
    # new exponent
    p = alpha + beta
    
    sq_diff_between_two_centres = np.linalg.norm(R_A-R_B)**2 # abs(R_A-R_B)**2 
    
    Normalization = ( (4*alpha*beta) /(np.pi**2) )** 0.75 
    
    K = Normalization * np.exp((-alpha*beta/p)*sq_diff_between_two_centres)
    
    Rp = (alpha*R_A + beta*R_B)/p
    
    return p, sq_diff_between_two_centres, K, Rp

In [46]:
def Overlap_integral(gaussA, gaussB):
    
    #integral of  (A|B) = ∫ dr1 g_1s(r_1 - R_{A}) g_1s(r_1 - R_{B})
    
    # equation (A8) pg 412 Szabo
    
    p, diff, K, Rp = Product_of_two_Gaussians(gaussA, gaussB)
    prefactor = (np.pi/p)**1.5
    return prefactor*K

In [47]:
def Kinetic_integral(gaussA, gaussB):
    
    #integral of  (A| -0.5 ∇^2 |B) = ∫ dr1 g_1s(r_1 - R_{A}) (-0.5 ∇^2)   g_1s(r_1 - R_{B})
    
    # equation (A11) pg 412 Szabo
    
    p, sq_diff_between_two_centres, K, Rp = Product_of_two_Gaussians(gaussA, gaussB)
    prefactor = (np.pi/p)**1.5
    
    alpha, R_A = gaussA
    beta, R_B = gaussB
    
    reduced_exponent = alpha*beta/p
    
    return reduced_exponent * ( 3- (2*reduced_exponent * sq_diff_between_two_centres) ) * prefactor * K

In [87]:
from scipy.special import erf
def F_zero_func(t):
    # pg 415 equation A32
    
    if t==0:
        return 1
    else:
        return np.sqrt(0.5*np.pi/t)* erf(t**0.5)
    

def Potential_integral(gaussA, gaussB, atomic_charge, atom_position):
    
    # nuclear attraction integral for 1s Gaussians
    
    # integral of  (A| -Z_{c}/r_{1C} |B) = ∫ dr1 g_1s(r_1 - R_{A}) (-Z_{c}/|r_{1} -R_{C}|)   g_1s(r_1 - R_{B})
    
    # equation (A16) pg 413 Szabo
    # final form is A33 pg 415!
    
    p, sq_diff_between_two_centres, K, Rp = Product_of_two_Gaussians(gaussA, gaussB)
    
    Rc = atom_position  # Position of atom C
    Zc = atomic_charge # charge of atom C
    
    
    int_val = (-2*np.pi*Zc/p) * K * F_zero_func(p*np.linalg.norm(Rp - Rc)**2)
    
    return int_val

In [113]:
def Product_of_four_Gaussians(GaussA, GaussB, GaussC, GaussD):
    
    # equation (A34) pg 415 Szabo
    
    #integral of  (AB|CD) = ∫ dr1 ∫ dr2 g_1s(r_1 - R_{A}) g_1s(r_1 - R_{B}) [r_{12}]^{-1}  g_1s(r_2 - R_{C}) g_1s(r_2 - R_{D})
    
        # note intgral over dr1 and dr2 with r_1 on LHS and r_2 on RHS
        # also see 1/r_{12} term!
    
    # final equation (A41) pg 416 Szabo
    
    p, sq_diff_between_two_centres_AB, K_AB, Rp = Product_of_two_Gaussians(GaussA, GaussB)
    q, sq_diff_between_two_centres_CD, K_CD, Rq = Product_of_two_Gaussians(GaussA, GaussB)


    multi_prefactor = (2*(np.pi**2.5)) / np.sqrt(p*q*(p+q))
    
    return multi_prefactor * K_AB * K_CD * F_zero_func((p*q*(p+q))*np.linalg.norm(Rp - Rq)**2)

# compute integral of AO basis!

- iterate through each of the atoms
- on each atom iterate through its orbitals
- for each orbital iterate through its three Gaussians (triple iteration)

(in this simple example we could just sum over the three Gaussians on each atom directly. This is due to each atom only having $1$ atomic orbital)

HOWEVER doing this explicitly will allow us to extend this program to solve more complicated molecules!

In [115]:
N_atoms = len(HeH_xyz)
atom_types = [atm[0] for atm in HeH_xyz]
atom_coord = [atm[1:] for atm in HeH_xyz]

In [175]:
# Initialize matrices

S = np.zeros((Basis_set_size, Basis_set_size))
T = np.zeros((Basis_set_size, Basis_set_size))
V = np.zeros((Basis_set_size, Basis_set_size))
multi_electron_tensor = np.zeros((Basis_set_size, Basis_set_size, Basis_set_size, Basis_set_size))

for atom_ind_A, atom_str_A in enumerate(atom_types): # iterate atoms
    
    # get charge and centre of atom
    Z_A = atomic_charges_dict[atom_str_A]
    R_A = atom_coord[atom_ind_A]
    
    # iterate through quantum numbers
    
    for n_shell_A in range(max_quantum_number_dict[atom_str_A]):
        
        
        # for each quantum number
        # get gaussian coefficients
        d_vec_shell_n = D[n_shell_A]
        
        # then scale exponents by zeta if necessary (pg 158 Szabo)
        # zeta would be stored in a dict
        zeta = 1
        alpha_vec_n_shell_A = alpha[n_shell_A]*zeta**2
        
        
        # iterate over the contraction coefficients
        for p in range(STOnG):
            
            # iterate through atoms once more
            for atom_ind_B, atom_str_B in enumerate(atom_types): # iterate atoms
                Z_B = atomic_charges_dict[atom_str_B]
                R_B = atom_coord[atom_ind_B]
                
                for n_shell_B in range(max_quantum_number_dict[atom_str_B]):
                    d_vec_shell_n = D[n_shell_B]
        
                    zeta = 1
                    alpha_vec_n_shell_B = alpha[n_shell_B]*zeta**2
                
                    for q in range(STOnG):
                        a = (atom_ind_A+1)*(n_shell_A+1)-1 # due to python indexing at zero
                        b = (atom_ind_B+1)*(n_shell_B+1)-1 # aka not 0s orbital, but 1s!
                        
                        
                        gaussA = (alpha_vec_n_shell_A[p], R_A)
                        gaussB = (alpha_vec_n_shell_A[q], R_B)
                        # get AO overlap
                        S[a,b] += D[n_shell_A, p] * D[n_shell_B, q]  * Overlap_integral(gaussA, gaussB)
                        
                        # kinetic ints
                        T[a,b] += D[n_shell_A, p] * D[n_shell_B, q]  * Kinetic_integral(gaussA, gaussB) 
                        
                        # Potential ints
                        for select_ind, selected_atom in enumerate(atom_types):
                            selected_atom_coords = atom_coord[select_ind]
                            atm_charge = atomic_charges_dict[selected_atom]
                            V[a,b] +=D[n_shell_A, p] * D[n_shell_B, q]  * Potential_integral(gaussA, gaussB,
                                                                                             atm_charge, selected_atom_coords)
                            
                        
                        
                        ## two more iterations over atoms for multi-electron-tensor (2e- or 4 body integrals)
                        for atom_ind_C, atom_str_C in enumerate(atom_types): # iterate atoms
                            Z_C = atomic_charges_dict[atom_str_C]
                            R_C = atom_coord[atom_ind_C]

                            for n_shell_C in range(max_quantum_number_dict[atom_str_C]):
                                d_vec_shell_n = D[atom_ind_C]

                                zeta = 1
                                alpha_vec_n_shell_C = alpha[n_shell_C]*zeta**2

                                for r in range(STOnG):
                                    
                                    for atom_ind_D, atom_str_D in enumerate(atom_types): # iterate atoms
                                        Z_D = atomic_charges_dict[atom_str_D]
                                        R_D = atom_coord[atom_ind_D]

                                        for n_shell_D in range(max_quantum_number_dict[atom_str_D]):
                                            d_vec_shell_n = D[n_shell_D]

                                            zeta = 1
                                            alpha_vec_n_shell_C = alpha[n_shell_C]*zeta**2

                                            for s in range(STOnG):
                                                c = (atom_ind_C+1)*(n_shell_C+1)-1 # due to python indexing at zero
                                                d = (atom_ind_D+1)*(n_shell_D+1)-1
                                                
                                                gaussC = (alpha_vec_n_shell_A[r], R_C)
                                                gaussD = (alpha_vec_n_shell_A[s], R_D)
                                                
                                                multi_electron_tensor[a,b,c,d]+= D[n_shell_A, p] * D[n_shell_B, q] * \
                                                                                D[n_shell_C, r] * D[n_shell_D, s] * \
                                                                                Product_of_four_Gaussians(gaussA, gaussB,
                                                                                                         gaussC, gaussD)

In [176]:
H_core = T + V

# Get orthogonal basis

In [177]:
## symmetric orthogonalization basis (Szabo pg 144)

eig_vals, U = np.linalg.eig(S)

diag_S = U.conj().T @ S @ U
diag_S_neg_half = np.diag(np.diag(diag_S)**(-0.5))

X = U @ diag_S_neg_half @ U.conj().T
X

array([[ 1.221177  , -0.43970998],
       [-0.43970998,  1.221177  ]])

In [178]:
from scipy.linalg import fractional_matrix_power
test = fractional_matrix_power(S, -0.5)
np.allclose(test, X)

True

# HF alg

In [195]:
# intial guess of C
P = np.zeros((Basis_set_size, Basis_set_size))


threshold=100
loop=0
while threshold>1e-6:
    
    # Get Fock matrix
    G = np.zeros((Basis_set_size, Basis_set_size))
    for i in range(Basis_set_size):
        for j in range(Basis_set_size):
            for x in range(Basis_set_size):
                for y in range(Basis_set_size):
                    G[i,j] += P[x,y]* (multi_electron_tensor[i,j,y,x] - 0.5*multi_electron_tensor[i,x,y,j])
                    #         ^^^^^^ note depends on P here!!!
    
    Fock = H_core + G
    
    # Get fock in ortU.conj().T @ S @ Uho basis
    FockPrime = X.conj().T @ Fock @ X
    
    eig_vals, C_prime = np.linalg.eig(FockPrime)

    #  order eig by size
    idx = eig_vals.argsort()
    eig_vals = eig_vals[idx]
    C_prime = C_prime[:, idx]
    
    # orthogonalize result in AO basis!
    C_ortho = X @ C_prime
    
    # build new density matrix
    P, P_old = 2* C_ortho @ C_ortho.conj().T, P
    
    # calculate threshold
    threshold = np.einsum('ij->', (P-P_old)**2)

print(eig_vals[0])
print(eig_vals[1])
P

-1.355199111754323
8.25066885448454


array([[ 3.36923624, -2.14785483],
       [-2.14785483,  3.36923624]])